In [1]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()
from goatools.obo_parser import GODag
obodag = GODag("go-basic.obo")

  EXISTS: go-basic.obo
  EXISTS: gene2go
go-basic.obo: fmt(1.2) rel(2020-01-01) 47,337 GO Terms


In [2]:
# how many gene in each GO term
# use human
from goatools.associations import read_ncbi_gene2go

go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], namespace='BP', go2geneids=True)
print("{N} GO terms associated with human NCBI Entrez GeneIDs".format(N=len(go2geneids_human)))

DEPRECATED read_ncbi_gene2go: USE Gene2GoReader FROM goatools.anno.genetogo_reader
DEPRECATED read_ncbi_gene2go CALLED FROM: <ipython-input-2-c14d79b3c135> BY <module>
HMS:0:00:06.921799 323,107 annotations READ: gene2go 
1 taxids stored: 9606
12285 IDs in loaded association branch, BP
12285 GO terms associated with human NCBI Entrez GeneIDs


In [3]:
len(go2geneids_human) # dictionary of each go term with the genes associated with it

12285

In [4]:
len(go2geneids_human['GO:0002576'])

124

In [5]:
def go2gene_count(go2geneids):
    '''
    count how many genes associate with each GO term at bottom level
    '''
    sizes = [len(go2geneids[term]) for term in go2geneids.keys()]
    return(dict(zip(go2geneids.keys(), sizes)))
count_dict = go2gene_count(go2geneids_human)

In [6]:
# test
count_dict['GO:0002576'] == len(go2geneids_human['GO:0002576'])

True

In [7]:
len(count_dict)

12285

In [8]:
# go-basic is acyclic
# parent = sum of child. 
# need to record the # of each child

In [10]:
# propagae

def count_gene(term, add, gene_associated = True):
    #print(term)
    if term not in aggr_count_dict.keys(): #if first time counted
        aggr_count_dict[term] = add # add the number to count dict
    elif gene_associated: # if new genes come in 
        aggr_count_dict[term] = aggr_count_dict[term] + add # add the genes
    
    # also add to parents
    try:
        parents = obodag[term]._parents
    except:
        parents = set()
        print(term, ' dont exist')
    for p in parents:
        if p not in aggr_count_dict.keys(): #if first time counted
            aggr_count_dict[p] = add
        else:
            aggr_count_dict[p] = aggr_count_dict[p] + add
        if p!= 'GO:0008150': # if not BP, root reachned
            count_gene(p, aggr_count_dict[p], gene_associated = False) # continue propagate to parent
    
    

In [12]:
aggr_count_dict = {}
count_gene('GO:0002576', count_dict['GO:0002576'])

In [13]:
aggr_count_dict

{'GO:0002576': 124,
 'GO:0045055': 124,
 'GO:0006887': 124,
 'GO:0016192': 124,
 'GO:0006810': 372,
 'GO:0051234': 744,
 'GO:0051179': 1240,
 'GO:0008150': 1984,
 'GO:0032940': 124,
 'GO:0046903': 124,
 'GO:0140352': 124,
 'GO:0009987': 124}

In [14]:
_ = [count_gene(term, count_dict[term]) for term in count_dict.keys()]
# seems some don't exist

GO:0060154  dont exist
GO:0061396  dont exist
GO:0052213  dont exist
GO:0072363  dont exist
GO:0052106  dont exist
GO:1901350  dont exist
GO:1902583  dont exist
GO:1990967  dont exist
GO:0072362  dont exist
GO:0098504  dont exist


In [ ]:
obodag['GO:0060154'] # not in?

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(aggr_count_dict, orient = 'index', columns = ['size'])
df.shape

In [ ]:
df.to_csv('human_bp_count.csv')